In [573]:
from twython import Twython, TwythonRateLimitError, TwythonError
from textblob import TextBlob
from glob import glob
from csv import DictReader, DictWriter
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
import pandas as pd
import os, re, string
import numpy as np
import emoji
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
import xml.etree.ElementTree as ET
import nltk
from nltk.classify import NaiveBayesClassifier, MaxentClassifier
from nltk.tag.sequential import ClassifierBasedPOSTagger

In [2]:
ConsumerKey = 'Xap2Jytwc3ehirzhWhPtIg' 
ConsumerSecret = 'M19t2uyoXSDnNxH01c4EAWvoAScpSyUlBHiGQx4U'
AccessToken = '166071132-r247QGhTdb9a1SdtH36avAndjAjwJksdJZLLqDYt'
AccessSecret = 'y7CGzzZzwyO72sx7yNedrS3hNXTAPWk5uqjDVdczykaKc'

twitter_user = 'HassanNisar'
tw = Twython(ConsumerKey, ConsumerSecret, AccessToken, AccessSecret)

In [154]:
def timeline(user_id, max_id=None):    
    for i in range(30):
        tweets = tw.get_user_timeline(screen_name=user_id,
                                      max_id=max_id,
                                      count=200,
                                      trim_user=True,
                                      exclude_replies=True,
                                      include_rts=False,
                                      tweet_mode='extended')
        if len(tweets) > 0:  # last page should have zero results
            for tweet in tweets:
                max_id = tweet['id'] - 1
                yield tweet
        else:
            break

In [222]:
# https://gist.github.com/emallson/9e56a99973b3091124cd
# I have a directory of csv files containing user profiles. 
# Pull the IDs from these.
fields = ['id', 'user', 'created_at', 'lang', 'full_text']
users_list = open("user.txt", "r")
for name in users_list:
    path = "timelines/{}.csv".format(name.rstrip())
    if os.path.exists(path):
        continue # skip users that have already been read.
    with open(path, "w", encoding='utf-8', newline='') as timecsv:
        print(name)
        writer = DictWriter(timecsv, fields)
        writer.writeheader()
        # max_id is used to continue a user's timeline from 
        # the last tweet read in the event that we get 
        # rate-limited in the middle of a user's timeline
        max_id = None
        while True:
            try:
                for tweet in timeline(name, max_id):
                    temp=tweet
                    sub = {k: v for k, v in tweet.items()
                           if k in fields}
                    sub['user'] = name.rstrip()
                    sub['full_text'] = sub['full_text'].rstrip()
                    max_id = sub['id']
                    if (sub['lang'] == 'ur'):
                        writer.writerow(sub)
                max_id = None
                break
            except TwythonRateLimitError as e:
                sleep_until(e.retry_after) # sleep until the given date
            except TwythonError as e:
                # I *think* this is caused by protected profiles. 
                # I can read some user profile info, but not the timeline
                print(e)
                print("Skipping...")

omar_journalist
('Connection aborted.', OSError("(10060, 'WSAETIMEDOUT')",))
Skipping...


In [662]:
# Reading All the user tweets
df = pd.concat([pd.read_csv(f, encoding='utf-8') for f in glob('timelines/*.csv')], ignore_index=True)

In [663]:
# Removing newline characters
df = df.replace(r'\n', '', regex = True)

In [664]:
# Removing weblinks
df['text'] = df['full_text'].apply(lambda x: re.sub(r'https?://[A-Za-z0-9./]+','',x))

In [665]:
# Removing English Language 
df['text'] = df['text'].apply(lambda x: re.sub(r'[a-z]','',x,flags=re.I))

In [666]:
# Removing Digits
df['text'] = df['text'].apply(lambda x: x.translate(str.maketrans('','',string.digits)))

In [667]:
# Removing Punctuations
df['text'] = df['text'].apply(lambda x: re.sub(r'[۔_:."-)(@#،,?!٪\'“”؟/]+\ *','',x))

In [668]:
# Removing Emojis
def give_emoji_free_text(text):
    allchars = [strs for strs in text]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

df['text'] = df['text'].apply(lambda x: give_emoji_free_text(x))

In [669]:
# Reading stopwords list
with open('stopwords.txt','r',encoding="utf8") as f:
    stopwords = f.readlines()
f.close()
stopwords = list(map(lambda s: s.strip(), stopwords))

In [670]:
# Removing Stopwords
def removeStopwords(x):
    s_list = [word for word in x.split() if word not in stopwords]
    str_ = ' '.join(s_list)
    return str_

df['text'] = df['text'].apply(lambda x: removeStopwords(x))

In [687]:
df['text'][2120]

'منظور پشتین کو بڑا جھٹکا سب سے زیادہ فعال رہنے والی’ گلا لئی اسماعیل ‘ اچانک لاپتہ ہوگئیپتہ کرنے کہاں پائی گئیحقیقت نے پختونوں'

In [686]:
df['full_text'][2120]

'منظور پشتین کو بڑا جھٹکا : سب سے زیادہ فعال رہنے والی’ گلا لئی اسماعیل ‘ اچانک لاپتہ ہوگئی، پتہ کرنے پر کہاں پائی گئی؟ حقیقت نے پختونوں کےhttps://t.co/goUTzxLFn3 https://t.co/goUTzxLFn3'

In [637]:
texts = df['text']
labels = df['user']

In [638]:
# split the dataset into training and validation datasets 
train_corpus, test_corpus, train_labels, test_labels = model_selection.train_test_split(texts, labels, train_size=0.75)

In [639]:
# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_labels)
test_y = encoder.fit_transform(test_labels)

In [716]:
def bow_extractor(corpus, ngram_range=(1,1)):
    vectorizer = CountVectorizer(analyzer='word', min_df=1, ngram_range=ngram_range, max_features=5000 )
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features

In [717]:
def tfidf_extractor(corpus, ngram_range=(1,1)):
    vectorizer = TfidfVectorizer(analyzer='word',
                                 min_df=1, 
                                 norm='l2',
                                 smooth_idf=True,
                                 use_idf=True,
                                 max_features=5000,
                                 ngram_range=ngram_range)
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features

In [718]:
# bag of words features
bow_vectorizer, bow_train_features = bow_extractor(train_corpus)  
bow_test_features = bow_vectorizer.transform(test_corpus)

In [719]:
# tfidf features
tfidf_vectorizer, tfidf_train_features = tfidf_extractor(train_corpus)  
tfidf_test_features = tfidf_vectorizer.transform(test_corpus)

In [720]:
bow_train_features.shape

(8961, 5000)

In [721]:
tfidf_train_features.shape

(8961, 5000)

In [646]:
def get_metrics(true_labels, predicted_labels):
    print('Accuracy:', np.round(metrics.accuracy_score(true_labels, predicted_labels), 2))
    print('Precision:', np.round(metrics.precision_score(true_labels, predicted_labels, average='weighted'), 2))
    print('Recall:', np.round(metrics.recall_score(true_labels, predicted_labels, average='weighted'), 2))
    print('F1 Score:', np.round(metrics.f1_score(true_labels, predicted_labels, average='weighted'), 2))

In [647]:
def train_predict_evaluate_model(classifier,train_features, train_labels,test_features, test_labels):
    # build model    
    classifier.fit(train_features, train_labels)
    # predict using model
    predictions = classifier.predict(test_features) 
    # evaluate model prediction performance   
    get_metrics(true_labels=test_labels, predicted_labels=predictions)
    return predictions

In [648]:
def confusionMatrix(test_labels, pred_labels):
    labels = np.unique(test_labels)
    cm =  metrics.confusion_matrix(test_labels, pred_labels, labels=labels)
    return pd.DataFrame(cm, index=labels, columns=labels)

In [722]:
mnb = MultinomialNB()
svm = SGDClassifier(loss='hinge', n_iter=100)

In [723]:
# Multinomial Naive Bayes with bag of words features
mnb_bow_predictions = train_predict_evaluate_model(classifier=mnb,
                                           train_features=bow_train_features,
                                           train_labels=train_labels,
                                           test_features=bow_test_features,
                                           test_labels=test_labels)

Accuracy: 0.83
Precision: 0.83
Recall: 0.83
F1 Score: 0.83


In [724]:
cm = confusionMatrix(test_labels, mnb_bow_predictions)
cm

,AftabIqbal_,HassanNisar,KhSaad_Rafique,mujibshami1,omar_journalist
AftabIqbal_,200,26,19,63,32
HassanNisar,11,739,7,11,14
KhSaad_Rafique,38,15,337,42,18
mujibshami1,35,24,5,543,25
omar_journalist,31,28,12,52,660


In [725]:
# Support Vector Machine with bag of words features
svm_bow_predictions = train_predict_evaluate_model(classifier=svm,
                                           train_features=bow_train_features,
                                           train_labels=train_labels,
                                           test_features=bow_test_features,
                                           test_labels=test_labels)

C:\Users\Nabeel\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


Accuracy: 0.8
Precision: 0.8
Recall: 0.8
F1 Score: 0.8


In [726]:
cm = confusionMatrix(test_labels, svm_bow_predictions)
cm

,AftabIqbal_,HassanNisar,KhSaad_Rafique,mujibshami1,omar_journalist
AftabIqbal_,168,34,35,62,41
HassanNisar,17,705,12,18,30
KhSaad_Rafique,34,8,341,37,30
mujibshami1,46,19,31,479,57
omar_journalist,20,11,21,37,694


In [727]:
# Multinomial Naive Bayes with tfidf features                                           
mnb_tfidf_predictions = train_predict_evaluate_model(classifier=mnb,
                                           train_features=tfidf_train_features,
                                           train_labels=train_labels,
                                           test_features=tfidf_test_features,
                                           test_labels=test_labels)

Accuracy: 0.79
Precision: 0.81
Recall: 0.79
F1 Score: 0.77


In [728]:
cm = confusionMatrix(test_labels, mnb_tfidf_predictions)
cm

,AftabIqbal_,HassanNisar,KhSaad_Rafique,mujibshami1,omar_journalist
AftabIqbal_,74,66,14,123,63
HassanNisar,0,758,1,9,14
KhSaad_Rafique,4,42,307,60,37
mujibshami1,3,45,0,538,46
omar_journalist,3,50,4,38,688


In [729]:
# Support Vector Machine with tfidf features
svm_tfidf_predictions = train_predict_evaluate_model(classifier=svm,
                                           train_features=tfidf_train_features,
                                           train_labels=train_labels,
                                           test_features=tfidf_test_features,
                                           test_labels=test_labels)

C:\Users\Nabeel\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


Accuracy: 0.83
Precision: 0.83
Recall: 0.83
F1 Score: 0.83


In [730]:
cm = confusionMatrix(test_labels,svm_tfidf_predictions)
cm

,AftabIqbal_,HassanNisar,KhSaad_Rafique,mujibshami1,omar_journalist
AftabIqbal_,176,38,29,65,32
HassanNisar,7,738,10,11,16
KhSaad_Rafique,30,13,345,43,19
mujibshami1,31,24,15,529,33
omar_journalist,17,21,16,38,691


In [278]:
df.groupby('user').describe()

id                                            \
                  count          mean           std           min   
user                                                                
AftabIqbal_      1364.0  9.919326e+17  5.477875e+16  8.930074e+17   
HassanNisar      3161.0  1.090922e+18  2.522257e+15  1.086237e+18   
KhSaad_Rafique   1779.0  9.554059e+17  8.530451e+16  6.165331e+17   
mujibshami1      2585.0  8.255037e+17  1.169837e+17  6.561426e+17   
omar_journalist  3059.0  1.033400e+18  3.395432e+16  9.843857e+17   

                                                                         
                          25%           50%           75%           max  
user                                                                     
AftabIqbal_      9.455906e+17  9.845359e+17  1.036725e+18  1.101880e+18  
HassanNisar      1.088754e+18  1.090928e+18  1.093094e+18  1.098899e+18  
KhSaad_Rafique   9.034022e+17  9.604295e+17  1.018891e+18  1.100772e+18  
mujibshami1      7.304463e+17  8.052898e+17  8.924360e+17  1.102089e+18  
omar_journalist  1.002215e+18  1.027607e+18  1.063122e+18  1.101543e+18

In [568]:
main_list = []

for path in glob('dataset/TRAIN/*.xml'):
    tree = ET.parse(path)
    root = tree.getroot()
    
    for s in root.iter('s'):
        child_list = []
        for child in s.getchildren():
            if (child.text != None):
                item = (child.text, child.attrib['pos'])
            if (len(item) == 2):
                child_list.append(item)
        
        if(child_list):
            main_list.append(child_list)

print('done !!!!')

done !!!!


In [569]:
test_list = []

for path in glob('dataset/TEST/*.xml'):
    tree = ET.parse(path)
    root = tree.getroot()
    
    for s in root.iter('s'):
        child_list = []
        for child in s.getchildren():
            if (child.text != None):
                item = (child.text, child.attrib['pos'])
            if (len(item) == 2):
                child_list.append(item)
        
        if(child_list):
            test_list.append(child_list)

print('done !!!!')

done !!!!


In [574]:
sent = 'بریکنگ نیوز : بڑا فیصلہ آ گیا ، پاکستان کو عالمی عدالت سے بڑی خوشخبری مل گئی'
u_tokens = nltk.word_tokenize(sent)

nbt_urdu = ClassifierBasedPOSTagger(train=main_list, classifier_builder=NaiveBayesClassifier.train)

print(nbt_urdu.evaluate(test_list))
print(nbt_urdu.tag(u_tokens))

0.8654746011839746
[('بریکنگ', 'PN'), ('نیوز', 'PN'), (':', 'PM'), ('بڑا', 'ADJ'), ('فیصلہ', 'NN'), ('آ', 'VB'), ('گیا', 'AA'), ('،', 'PM'), ('پاکستان', 'PN'), ('کو', 'P'), ('عالمی', 'ADJ'), ('عدالت', 'NN'), ('سے', 'SE'), ('بڑی', 'ADJ'), ('خوشخبری', 'ADJ'), ('مل', 'VB'), ('گئی', 'AA')]


In [732]:
# def addPOSTag(x):
#     s_list = []
#     tag = nbt_urdu.tag(x)
#     for s in tag:
#         word, tag = s
#         s_list.append(tag)
        
#     str_ = ' '.join(s_list)
#     return str_

# df['POS'] = df['text'].apply(lambda x: addPOSTag(x))

In [601]:
texts = df['POS']
labels = df['user']

In [617]:
texts

0        PN PN PN  A EXP  PN  CC NN  PM  MUL FR U AD FR...
1        PN NN PN PN PN CA EXP FR ADJ  EXP  MUL FR MUL ...
2        AP CA CA PN ADV PN PN  PN  AA TA  PN PN  EXP  ...
3        PN PN VB SM AA AP CA AD EXP  CA PN PN CA EXP C...
4        CA PN U A EXP  ADJ  AD MUL ADV  EXP  ADV  MUL ...
5        PD PN PN PN  CA U EXP  PN CC CA ADJ EXP REP G ...
6        CC EXP PN EXP PN EXP ADV EXP MUL EXP  CA EXP  ...
7        AP CA CA PN ADV PN PN  CA PN PN PN  EXP  MUL A...
8        AP CA CA PN ADV PN PN  PN  AA TA  PN PN  EXP  ...
9        CA PN U EXP  CC NN  PN AP EXP INT PN PN PN PN ...
10       AP CA CA PN ADV PN PN  PN  AA TA  PN PN  EXP  ...
11       AP CA SE EXP U CA U PN  AD EXP  AP MUL AA EXP ...
12       CC PD EXP CA U CA PN U PN PN  EXP  MUL AA CA P...
13       PD PN EXP PN CC PN PN PN  CA AD U A MUL AA EXP...
14       AP CA CA PN ADV SM CA CC EXP PN CA EXP CA CA P...
15       PP G CC CA PN CC VB PM  MUL FR I  U FR VB KER ...
16       CA PN U CA EXP  FR INT EXP  I  INT INT EXP  AD.